IMPORTAR BIBLIOTECAS

In [79]:
import pandas as pd
import numpy as np
import xlwings as xw

LENDO DADOS

In [26]:
def ler_arquivo(nome_arquivo):
    try:
         df =pd.read_excel(f'bases/{nome_arquivo}.xlsx')
         return df
    except FileNotFoundError as e:
        return f'Erro:{e}'

        #Aula 07\bases\dados de entregas.xlsx

In [33]:
df = ler_arquivo ('dados de entregas')

df.head()


,id_entrega,filial,cliente,status,descricao_entrega,tipo_servico,valor_frete,data_postagem,prazo_previsto,data_prevista,data_entrega,dias_atraso
0,3619,SC,Americanas,ENTREGUE,Extravio temporário,ECONÔMICO,25.16,2026-01-09 20:43:34.883,5,2026-01-14 20:43:34.883,2026-01-15 20:43:34.883,1
1,4850,RJ,Amazon,ENTREGUE,Entrega reagendada,NORMAL,38.17,2026-01-06 20:43:34.889,5,2026-01-11 20:43:34.889,2026-01-10 20:43:34.889,0
2,3232,MG,Magazine Luiza,EM TRÂNSITO,Cliente ausente no local,NORMAL,311.62,2026-02-03 20:43:34.881,1,2026-02-04 20:43:34.881,2026-02-05 20:43:34.881,1
3,3944,RS,Amazon,EM TRÂNSITO,Cliente ausente no local,NORMAL,65.46,2026-01-17 20:43:34.885,7,2026-01-24 20:43:34.885,2026-01-26 20:43:34.885,2
4,2633,SP,Amazon,EM TRÂNSITO,Problema operacional na rota,ECONÔMICO,104.62,2026-02-03 20:43:34.875,2,2026-02-05 20:43:34.875,2026-02-05 20:43:34.875,0


SLA

In [85]:
#Total de entregas sem atraso

sla_ok = df.loc[df['dias_atraso']==0].shape[0]
sla_ok

#Total de entregas com atraso
sla_fora = df.loc[df['dias_atraso']!=0].shape[0]
sla_fora

#Total de entregas
total_entregas = df.loc[df['status']=='ENTREGUE'].shape[0]
total_entregas

#perfomance
performance = (sla_ok/total_entregas)*100
f'{round(performance,2)}% das entregas foram deitas dentro do prazo.'

print(f'Total entregas      |{total_entregas}')
print(f'Entregas no prazo   |{sla_ok}')
print(f'entregas fora prazo |{sla_fora}')
print(f'Perfomance          |{round(performance,2)}% das entregas foram deitas dentro do prazo.')


Total entregas      |2503
Entregas no prazo   |1467
entregas fora prazo |3733
Perfomance          |58.61% das entregas foram deitas dentro do prazo.


KPI - ENTREGAS ATRASADAS

In [76]:
#total_entregas = df.loc[df['status']=='ENTREGUE'].shape[0]

entregas_atrasadas = df.loc[
    (df['status']=='ENTREGUE') & (df['dias_atraso'] > 0)
].shape[0]

percentual_atraso = (entregas_atrasadas / total_entregas) * 100
percentual_atraso = f'{round(percentual_atraso, 1)}%'

percentual_atraso


'62.8%'

SLA - IDENTIFICANDO ONDE AS OPERACOES FUNCIONAM MELHOR

In [89]:
#sla_filial = df_total_entregas.groupby('filial')['dias_atraso'].mean()

# filtrar somente entregas concluídas
df_ent = df.loc[df['status'] == 'ENTREGUE'].copy()

sla_por_filial = (
    df_ent
    .assign(em_atraso=lambda x: x['dias_atraso'].fillna(0) > 0)
    .groupby('filial')
    .agg(
        total_entregas=('id_entrega', 'count'),
        entregas_atrasadas=('em_atraso', 'sum')
    )
    .assign(
        perc_atraso=lambda x: np.where(
            x['total_entregas'] > 0,
            (x['entregas_atrasadas'] / x['total_entregas']) * 100,
            0.0
        ),
        sla_pct=lambda x: 100 - x['perc_atraso']
    )
    .assign(
        perc_atraso=lambda x: x['perc_atraso'].round(1),
        sla_pct=lambda x: x['sla_pct'].round(1)
    )
    .reset_index()
    .sort_values('sla_pct', ascending=False)
)

sla_por_filial.reset_index(names='%Entregas no prazo')

,%Entregas no prazo,filial,total_entregas,entregas_atrasadas,perc_atraso,sla_pct
0,2,RJ,385,223,57.9,42.1
1,0,MG,366,226,61.7,38.3
2,1,PR,446,275,61.7,38.3
3,5,SP,425,273,64.2,35.8
4,3,RS,438,285,65.1,34.9
5,4,SC,443,289,65.2,34.8


In [ ]:
sla_filial = df_total_entregas.groupby('filial')['dias_atraso'].apply(
    lambda grupo:(grupo ==0).mean()*100
).sort_values(ascending=False)

sla_filial